In [3]:
import os
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from numpy import linalg
import DAG_lib as dag
from functools import partial
from concurrent.futures import ProcessPoolExecutor
import random
from scipy import stats
import validation as val

In [5]:
with open("../Transfusion Training data_20240311.pkl", 'rb') as pickle_file:
    data = pickle.load(pickle_file)
    
T = np.array(data["T"])
w,d,input_Xt,input_Y,_ = dag.load_data(data)
modes = stats.mode(input_Xt, axis=0)
training_modes = modes.mode[1:]

data loaded


In [6]:
with open("../Transfusion Validation data_20240311.pkl", 'rb') as pickle_file:
    data = pickle.load(pickle_file)
    
T = np.array(data["T"])
w,d,input_Xt,input_Y,_ = dag.load_data(data)

data loaded


In [11]:
folder = "../fine_tune_lambda_3"
theta_whole_linear_list = [folder+"/"+name for name in list(os.listdir(folder)) if ".npy" in name]
final_dict = val.parallel_process_files(theta_whole_linear_list, folder, w, d, input_Xt, input_Y, T, training_modes)

100%|██████████| 11/11 [00:00<00:00, 37.02it/s]


In [65]:
import pandas as pd
error_df = pd.DataFrame()
for theta_whole_linear_dir in theta_whole_linear_list:
    my_lambda = theta_whole_linear_dir.replace(folder+'/theta_whole_linear_lambda_', '')
    my_lambda = my_lambda.replace('.npy', '')
    median_error = []
    for feature in final_dict[my_lambda].keys():
        data = final_dict[my_lambda][feature]
        values_list = [value for sublist in data.values() for value in sublist]
        median_error.append(np.median(values_list))
        
    theta_whole_linear = np.load('theta_whole_linear_lambda_'+my_lambda+'.npy')
    A_fitted_linear = theta_whole_linear[:,1:]
    step2_dict = dag.cycle_find(A_fitted_linear,w,d)
    data = step2_dict["all_closed_walk_num"]
    median_error.extend([data[2],data[3],data[4]])
    error_df[my_lambda] = median_error

In [66]:
error_df

,3.162277660168379e-06,1e-05,0.01,0.001,3.162277660168379e-07,1e-07,3.1622776601683795e-05,0.0031622776601683794,0.0001,0.00031622776601683794,1e-06
0,19.047270,31.393449,6.907755e+01,6.907755e+01,17.099742,17.128695,310.852586,6.907755e+01,69.077553,69.077553,17.435442
1,15.438459,15.595759,4.490041e+02,4.490041e+02,15.488676,15.486908,14.857646,4.490041e+02,33.477730,449.004093,15.481271
2,8.172475,8.058153,3.108490e+02,3.890683e+00,8.266914,8.278576,7.123116,3.108490e+02,3.500524,3.796122,8.238726
3,11.923586,12.002460,4.662787e+02,4.662787e+02,11.964059,11.966177,12.286832,4.662787e+02,11.738218,24.808958,11.954210
4,9.464722,9.073980,1.309897e+01,1.017720e+01,9.637479,9.627096,8.068967,1.309897e+01,3.445570,3.652608,9.584497
5,17.013634,15.114977,1.037876e+02,1.042779e+02,17.255176,17.235742,13.886145,1.037876e+02,17.612962,103.787605,17.340018
6,12.393164,12.133653,2.298162e-14,2.298162e-14,12.334981,12.324785,10.729120,2.298162e-14,8.416612,69.077553,12.348215
7,13.842902,13.748168,5.871728e+02,5.871728e+02,14.007744,14.020588,13.815513,5.871728e+02,12.350649,587.172792,13.945977
8,15.433920,15.521951,3.108526e+02,3.108526e+02,15.486697,15.509535,15.590042,3.108526e+02,18.715616,310.852586,15.440214
9,10.247620,9.747839,4.843185e+00,4.859034e+00,10.482808,10.491096,9.202485,4.843185e+00,8.827144,4.585699,10.426874


In [68]:
error_df.transpose().to_excel("With_different_lambda.xlsx")

In [12]:
my_lambda = val.find_best_lambda(final_dict)

In [13]:
my_lambda

'3.162277660168379e-06'

## See if there are still cycles in the step 2 results

In [33]:
### step 2: find cycles in the fitted graph
import itertools

theta_whole_linear = np.load('theta_whole_linear_lambda_'+'3.162277660168379e-06'+'.npy')

A_fitted_linear = theta_whole_linear[:,1:]
step2_dict = dag.cycle_find(A_fitted_linear,w,d)

In [34]:
step2_dict["all_closed_walk_num"]

{2: 188, 3: 1131, 4: 10096}

In [34]:
{2: 146, 3: 492, 4: 3772}

{2: 0, 3: 0, 4: 0}

## Generate random theta whole within range to compare performance

In [20]:
theta_whole_linear_list = [folder+"/"+name for name in list(os.listdir(folder)) if ".npy" in name]
all_theta_whole = []
min_map = np.zeros([20,21])
max_map = np.zeros([20,21])

for i in tqdm(range(20)):
    for j in range(21):
        min_num = 10000
        max_num = 0
        for theta_whole_linear_dir in theta_whole_linear_list:
            matrix = np.load(theta_whole_linear_dir)
            if matrix[i,j] < min_num:
                min_num = matrix[i,j]
            if matrix[i,j] > max_num:
                max_num = matrix[i,j]
        min_map[i,j] = min_num
        max_map[i,j] = max_num

random.seed(42)
theta_whole = np.zeros([20,21])
for i in tqdm(range(20)):
    for j in range(21):
        random_number = random.uniform(min_map[i,j], max_map[i,j])
        theta_whole[i,j] = random_number

# theta_whole
loss = []
# Generate prediction result
y_pred = g(input_Xt @ theta_whole.T)

# Calculate Metric for each column
for i in range(y_pred.shape[1] - 1):
    start = 0
    y_score = np.array(normalize_array(y_pred[:, i]))
    for pat_id, length in enumerate(T-1):
        log_loss_value = my_log_loss(input_Y[:, i][start:start+length], y_score[start:start+length])
        start += length
    loss.append(log_loss_value)
loss

100%|██████████| 20/20 [00:00<00:00, 46474.28it/s]


[17.722156737705276,
 39.1597447003262,
 64.27855891431345,
 60.459830845804454,
 54.62510466391544,
 31.836777585666237,
 38.37303815858389,
 51.77474230851244,
 48.19459262574705,
 21.12082028335391,
 22.058761196594116,
 17.916324990037143,
 58.38067553117424,
 48.70745840347513,
 16.590872019816633,
 53.81861417455831,
 23.800912620956183,
 16.769044024349725,
 51.32877711881935]